In [2]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 4.7 MB/s eta 0:00:00


In [30]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.2 MB/s eta 0:00:00


In [31]:
import pandas as pd
import numpy as np

import optuna
from sklearn.preprocessing import LabelEncoder
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

import warnings
warnings.filterwarnings(action="ignore")

In [12]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [13]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID'])

In [14]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [15]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])

In [16]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
test_x = test_x.to_numpy()

In [23]:
model = TabNetClassifier(verbose=20, optimizer_fn=torch.optim.AdamW)

In [24]:
model.fit(train_x, train_y, max_epochs=100)

epoch 0  | loss: 1.38951 |  0:00:01s
epoch 20 | loss: 0.85267 |  0:00:28s
epoch 40 | loss: 0.78693 |  0:00:54s
epoch 60 | loss: 0.75869 |  0:01:21s
epoch 80 | loss: 0.75723 |  0:01:48s


In [25]:
preds = model.predict(test_x)

In [26]:
submit = pd.read_csv("./sample_submission.csv")

In [27]:
submit['Y_Class'] = preds

In [28]:
submit.to_csv('./sumbission.csv', index=False)